# Notes #
1. Data preparation
    + Steps
        - Load svm_light file
        - Convert to array or dataframe
        - Check for missing values
        - Categorical fields to numerical
        - Class distribution
        - Normalize if necessary
        - Test, train split


**Rough notes**
1. Options for 6th,7th and 8th datasets
    + Url dataset -- Loading issue, SVM light format
    + Gas sensor -- Got to checkout
    + 1-2 basic datasets 
    + Generate artificial datasets
    
2. Notes on URL dataset
    + What are these 3.2 million features?
    + What's the structure of this svm_light file? The data returned.
    + Unable to extract the contents of file "FeatureTypes"
    + http://www.sysnet.ucsd.edu/projects/url/
    + What is a sparse dataset?
    
**3. Keeping aside for the time being as it is taking too long  
to load the dataset.(Shape - 16k,3.2 million)**

---

**Data preparation**

In [8]:
# Data preparation #

# Load CSV using Pandas
import pandas as pd
import numpy as np
# filename = 'adult.data1.csv'

from sklearn.datasets import load_svmlight_file
 
X, y = load_svmlight_file("../data/Day0.svm")
print(type(X))
X_train_pandas  = pd.SparseDataFrame(X_train)
# y_train_pandas  = pd.SparseDataFrame(pd.SparseSeries(y_train))

# names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
#          'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
#          'hours-per-week', 'native-country', 'target']
# data = pd.read_csv("dataset.csv", delimiter=" ", header=None, names=names)

# Check for columns that contain missing values #
# col_names = data.columns
# print(col_names)
# num_data = data.shape[0]
# for c in col_names:
#     num_non = data[c].isin(["?"]).sum()
#     if num_non > 0:
#         print (c)
#         print (num_non)
#         print ("{0:.2f}%".format(float(num_non) / num_data * 100))
#         print ("\n")

# data = data[data["workclass"] != "?"]
# data = data[data["occupation"] != "?"]
# data = data[data["native-country"] != "?"]

# # Convert categorical fields #
# categorical_col = ['workclass', 'education', 'marital-status', 'occupation',
#                    'relationship', 'race', 'sex', 'native-country', 'target']

# # for col in categorical_col:
# #     data.col
    
# # for col in categorical_col:
# #     categories = unique_of(data.col)
# #     num_cat = count(categories)
# #     for cat in categories:
# #         data.col[cat] = index_of(cat in categories)

# for col in categorical_col:
#     b, c = np.unique(data[col], return_inverse=True)
#     data[col] = c

# feature_list = names[:14]
# # Test train split #
# X = data.loc[:, feature_list]
# Y = data[['target']]

# # Split the dataset into test and train datasets

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

# data.shape
# data.head()


<class 'scipy.sparse.csr.csr_matrix'>


KeyboardInterrupt: 

---

**Keras Model**

In [ ]:
# Keras #

### Logistic regression using DNN ###
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

np.random.seed(7)

# Defining model #

model = Sequential()
model.add(Dense(1,input_dim=14,activation='sigmoid'))

# Compile the model #

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model #

model.fit(x_train, y_train, epochs=800, batch_size=100, verbose=0)

# Evaluate the model #

scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

---

**Scikit model**

In [ ]:
# Scikit learn #

# Import and create an instance of your model(Logistic regression)

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

# Train your model using the training dataset

logisticRegr.fit(x_train,y_train)

# Predict the output 

predictions = logisticRegr.predict(x_test)
print(predictions)

score = logisticRegr.score(x_test,y_test)
print(score)


---

**Kfold validation**

In [ ]:
# Validation #
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

seed = 7
numpy.random.seed(seed)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X, Y):
    # create model
    model = Sequential()
    model.add(Dense(1,input_dim=14,activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X.iloc[train], Y.iloc[train], epochs=500, batch_size=100, verbose=0)
    # evaluate the model
    scores = model.evaluate(X.iloc[test], Y.iloc[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


---

**ROC Curve and Confusion matrix**

In [ ]:
# Prediction #
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
# y_pred = (y_pred>0.5)
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


# Creating the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

---

**Classification Report**

In [ ]:
from sklearn.metrics import classification_report

target_names = ['<=50K', '>50']
print(classification_report(y_test, y_pred, target_names=target_names))

---

# Pending #
1. See if you can fine tune keras model(75% acc) to match scikit model(78% acc)
2. Exploratory analysis of the dataset
3. Latex explanations
4. **Cross check what's wrong with ROC  
and Confusion matrix**